In [2]:
from flask import Flask, render_template, request
from joblib import load
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
import _pickle as pickle
from random import sample
from scipy.stats import halfnorm

In [ ]:
 
with open("..\\data\\refined_cluster.pkl", 'rb') as fp:
    cluster_df = pickle.load(fp)
    
with open("..\\data\\vectorized_refined.pkl", 'rb') as fp:
    vect_df = pickle.load(fp)

In [22]:
cluster_df.head()

,Bios,Movies,Religion,Music,Politics,Social Media,Sports,Age,Cluster #
0,Typical twitter fanatic. Infuriatingly humble ...,Thriller Action,Buddhist,Gospel Pop Rock,Progressive,Twitter Reddit Instagram,Basketball Football Hockey,19,1
1,Web junkie. Analyst. Infuriatingly humble intr...,Comedy Drama,Spiritual,HipHop Pop,Conservative,Twitter Facebook,Basketball Soccer Hockey,19,0
2,Avid web maven. Food practitioner. Gamer. Twit...,Horror Comedy,Christian,HipHop Rock,Moderate,Reddit Youtube,Baseball Football,21,1
3,Twitteraholic. Extreme web fanatic. Food buff....,Adventure Comedy,Agnostic,Rock,Moderate,Facebook Youtube,Basketball Football Hockey,25,1
4,Bacon enthusiast. Falls down a lot. Freelance ...,Action Adventure Drama,Hindu,Latin Rock HipHop,Conservative,Twitter TikTok Youtube,Hockey Football Basketball,19,0


In [23]:
vect_df.head()

,Age,advocate,aficionado,alcohol,alcoholaholic,amateur,analyst,animals,apathy,avid,...,tiktok,twitter,youtube,baseball,basketball,football,hockey,other,soccer,Cluster #
0,0.032258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1
1,0.032258,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0
2,0.096774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1
3,0.225806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1
4,0.032258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


In [24]:
def rename(temp):
    temp['cluster'] = temp['Cluster #']
    temp = temp.drop(columns='Cluster #')
    return temp

In [25]:
cluster_df = rename(cluster_df)

In [26]:
cluster_df

,Bios,Movies,Religion,Music,Politics,Social Media,Sports,Age,cluster
0,Typical twitter fanatic. Infuriatingly humble ...,Thriller Action,Buddhist,Gospel Pop Rock,Progressive,Twitter Reddit Instagram,Basketball Football Hockey,19,1
1,Web junkie. Analyst. Infuriatingly humble intr...,Comedy Drama,Spiritual,HipHop Pop,Conservative,Twitter Facebook,Basketball Soccer Hockey,19,0
2,Avid web maven. Food practitioner. Gamer. Twit...,Horror Comedy,Christian,HipHop Rock,Moderate,Reddit Youtube,Baseball Football,21,1
3,Twitteraholic. Extreme web fanatic. Food buff....,Adventure Comedy,Agnostic,Rock,Moderate,Facebook Youtube,Basketball Football Hockey,25,1
4,Bacon enthusiast. Falls down a lot. Freelance ...,Action Adventure Drama,Hindu,Latin Rock HipHop,Conservative,Twitter TikTok Youtube,Hockey Football Basketball,19,0
...,...,...,...,...,...,...,...,...,...
6595,Typical pop culture nerd. Infuriatingly humble...,Thriller Comedy Action,Catholic,Pop Rock HipHop,Liberal,Facebook Reddit Youtube,Baseball,28,1
6596,Avid web junkie. Lifelong alcohol guru. Hardco...,Action Horror,Spiritual,Rock Classical,Conservative,Facebook Instagram Youtube,Baseball Hockey,24,1
6597,Music ninja. Bacon fanatic. Reader. Total comm...,Adventure Drama,Catholic,Country HipHop,Progressive,Facebook Youtube,Football Basketball,24,1
6598,Communicator. Bacon lover. Award-winning intro...,Adventure Drama,Christian,Country Pop Rock,Moderate,Pinterest Facebook Reddit,Baseball Football,20,1


In [27]:
vect_df = rename(vect_df)
vect_df.head()

,Age,advocate,aficionado,alcohol,alcoholaholic,amateur,analyst,animals,apathy,avid,...,tiktok,twitter,youtube,baseball,basketball,football,hockey,other,soccer,cluster
0,0.032258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1
1,0.032258,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0
2,0.096774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1
3,0.225806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1
4,0.032258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0


In [30]:
  
with open("..\\data\\refined_cluster.pkl", 'wb') as fp:
    pickle.dump(cluster_df, fp)
    
with open("..\\data\\vectorized_refined.pkl", 'wb') as fp:
    pickle.dump(vect_df, fp)